# Homework 1 - Berkeley STAT 157

Handout 1/22/2017, due 1/29/2017 by 4pm in Git by committing to your repository. Please ensure that you add the TA Git account to your repository.

1. Write all code in the notebook.
1. Write all text in the notebook. You can use MathJax to insert math or generic Markdown to insert figures (it's unlikely you'll need the latter). 
1. **Execute** the notebook and **save** the results.
1. To be safe, print the notebook as PDF and add it to the repository, too. Your repository should contain two files: ``homework1.ipynb`` and ``homework1.pdf``. 

The TA will return the corrected and annotated homework back to you via Git (please give `rythei` access to your repository).

In [1]:
from mxnet import ndarray as nd

## 1. Speedtest for vectorization

Your goal is to measure the speed of linear algebra operations for different levels of vectorization. You need to use `wait_to_read()` on the output to ensure that the result is computed completely, since NDArray uses asynchronous computation. Please see http://beta.mxnet.io/api/ndarray/_autogen/mxnet.ndarray.NDArray.wait_to_read.html for details. 

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $4096 \times 4096$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. 
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time.
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time.
1. Bonus question - what changes if you execute this on a GPU?

In [ ]:
import time

# 1
A = nd.random.normal(loc=0, scale=1, shape=[4096, 4096])
B = nd.random.normal(loc=0, scale=1, shape=[4096, 4096])

# 2
# t = time.time()
# C = nd.dot(A, B)
# C.wait_to_read()
# print(time.time() - t)

# # 3
# t = time.time()
# columns = [nd.dot(A, B[:, i]).reshape(4096, 1) for i in range(4096)]
# C = nd.concat(*columns, dim=1)
# C.wait_to_read()
# print(time.time() - t)

# 4
t = time.time()
C = nd.empty(shape=[4096, 4096])
for i in range(4096):
    for j in range(4096):
        C[i, j] = nd.dot(A[i, :], B[:, j])
C.wait_to_read()
print(time.time() - t)

## 2. Semidefinite Matrices

Assume that $A \in \mathbb{R}^{m \times n}$ is an arbitrary matrix and that $D \in \mathbb{R}^{n \times n}$ is a diagonal matrix with nonnegative entries. 

1. Prove that $B = A D A^\top$ is a positive semidefinite matrix. 
1. When would it be useful to work with $B$ and when is it better to use $A$ and $D$?

1. Let $x$ be a m-dimensional vector, and let $C$ be a diagonal matrix containing the square roots of the diagonal entries in $D$ (so $C^2 = D$). Then $x^TBx = x^TADA^Tx = \|CA^Tx\|^2 >= 0$, and this completes the proof that B is positive semidefinite.

2. It would be useful to work with A and D if you're doing computations that may require the square root of B. Otherwise, just using B by itself would be faster.

## 3. MXNet on GPUs

1. Install GPU drivers (if needed)
1. Install MXNet on a GPU instance
1. Display `!nvidia-smi`
1. Create a $2 \times 2$ matrix on the GPU and print it. See http://d2l.ai/chapter_deep-learning-computation/use-gpu.html for details.

## 4. NDArray and NumPy 

Your goal is to measure the speed penalty between MXNet Gluon and Python when converting data between both. We are going to do this as follows:

1. Create two Gaussian random matrices $A, B$ of size $4096 \times 4096$ in NDArray. 
1. Compute a vector $\mathbf{c} \in \mathbb{R}^{4096}$ where $c_i = \|A B_{i\cdot}\|^2$ where $\mathbf{c}$ is a **NumPy** vector.

To see the difference in speed due to Python perform the following two experiments and measure the time:

1. Compute $\|A B_{i\cdot}\|^2$ one at a time and assign its outcome to $\mathbf{c}_i$ directly.
1. Use an intermediate storage vector $\mathbf{d}$ in NDArray for assignments and copy to NumPy at the end.

In [31]:
import numpy as np
A = nd.random.normal(loc=0, scale=1, shape=[4096, 4096])
B = nd.random.normal(loc=0, scale=1, shape=[4096, 4096])

t = time.time()
c = np.zeros(4096)
for i in range(4096):
    c[i] = nd.norm(nd.dot(A, B[i]), ord=2).asscalar()
print(time.time() - t)
    
t = time.time()
d = nd.zeros(4096)
for i in range(4096):
    d[i] = nd.norm(nd.dot(A, B[i]), ord=2)
d = d.asnumpy()
print(time.time() - t)

32.971606731414795
38.44051194190979


## 5. Memory efficient computation

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

In [44]:
nd.elemwise_add(nd.multiply(A, B), C, out=C)


[[ -36.82398224   43.56594086   57.57830048 ...,  -87.15515137
    -1.04497373  -30.40262985]
 [  29.85430145   59.5067749    59.79814529 ...,   21.76594734
    95.20579529  -42.27321625]
 [  -7.26595116 -101.27383423   55.39715576 ...,  -74.39221954
   102.45467377    8.85462379]
 ..., 
 [-117.63174438   62.79365158 -113.39180756 ...,  -41.78181458
     4.53057957   15.69922924]
 [  25.4855442    -2.86085081    5.29161549 ...,   74.39033508
    -5.14929104  -58.45245361]
 [-131.14347839  -48.33370972   -0.24792343 ...,  -12.27658749    4.3858757
   -29.27905083]]
<NDArray 4096x4096 @cpu(0)>

## 6. Broadcast Operations

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 20$ and $x = \{-10, -9.9, \ldots 10\}$. Implement code that generates such a matrix.

In [41]:
x = nd.arange(-10, 11).reshape(21, 1)
js = nd.arange(1, 21)
A = nd.power(x, js)
A


[[ -1.00000000e+01   1.00000000e+02  -1.00000000e+03   1.00000000e+04
   -1.00000000e+05   1.00000000e+06  -1.00000000e+07   1.00000000e+08
   -1.00000000e+09   1.00000000e+10  -9.99999980e+10   9.99999996e+11
   -9.99999983e+12   1.00000000e+14  -9.99999987e+14   1.00000003e+16
   -9.99999984e+16   9.99999984e+17  -9.99999998e+18   1.00000002e+20]
 [ -9.00000000e+00   8.10000000e+01  -7.29000000e+02   6.56100000e+03
   -5.90490000e+04   5.31441000e+05  -4.78296900e+06   4.30467200e+07
   -3.87420480e+08   3.48678451e+09  -3.13810596e+10   2.82429522e+11
   -2.54186593e+12   2.28767931e+13  -2.05891136e+14   1.85302015e+15
   -1.66771819e+16   1.50094642e+17  -1.35085174e+18   1.21576651e+19]
 [ -8.00000000e+00   6.40000000e+01  -5.12000000e+02   4.09600000e+03
   -3.27680000e+04   2.62144000e+05  -2.09715200e+06   1.67772160e+07
   -1.34217728e+08   1.07374182e+09  -8.58993459e+09   6.87194767e+10
   -5.49755814e+11   4.39804651e+12  -3.51843721e+13   2.81474977e+14
   -2.25179981e+1